In [11]:
# 서버 카프카에 "image_topic" (base64 ecoding된 이미지) 토픽 추가 하였다, 없으면 추가

#setting
import os
os.chdir("..")
import nb_init

os.environ['CEP_ENV'] = 'local'

In [12]:
# 필요 Pyspark 모듈 import
from cep_common import spark_utils as su
from cep_common import config as con

from pyspark.sql.types import *
import pyspark.sql.functions as F

# Create SparkSession
spark = su.create_spark_session(appName="hdfs-sample-delta")

In [13]:
file_name = "example.csv"
src_path = "hdfs://{}/datalake/hdfs/test/{}".format(con.HADOOP_NODE, file_name)

In [14]:
# HDFS에서 CSV 데이터를 읽어온다
csv_df_load = spark.read \
    .format("csv") \
    .load(src_path)

In [15]:
csv_df_load.show()

+------+---+
|   _c0|_c1|
+------+---+
|Second|  2|
|Fourth|  4|
| First|  1|
| Third|  3|
| Fifth|  5|
+------+---+



In [16]:
# elasticsearch 노드 정보
es_conf = {
    "es.nodes": con.ES_NODES
}

In [17]:
# elasticsearch 인덱싱
csv_df_load.write \
    .format("es") \
    .options(**es_conf) \
    .mode("overwrite") \
    .save("example/_doc")

In [18]:
spark.stop()

# 포스트맨에서 쿼리 확인

GET http://tib.t3q.ai:9200/_search
```
Header:  
    Content-Type: application/x-ndjson  

Body:  
    JSON:   
{
    "query": {
        "term": {
           "_index": {
              "value": "example"
           }
        }
    }

}
```
